# Proyecto Clase - Análisis de homogeneidad covariables formadoras de suelo
## Preparacion de datos

1. Cargar las capas de archivos shapefile de la cuenca curití
2. Cargar la imagen sentinel 2 de la cuenca
3. Cargar el DEM Alos Palsar de colombia

In [11]:
import sys
sys.path.append("../src")
import data_preprocessing
data_preprocessing.carga_shp_raster_proj()


Datos del DEM
 {'driver': 'GTiff', 'dtype': 'int16', 'nodata': 0.0, 'width': 124430, 'height': 157773, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Marco_Geocentrico_Nacional_de_Referencia_Transverse_Mercator",GEOGCS["GCS_Marco_Geocentrico_Nacional_de_Referencia",DATUM["Marco_Geocentrico_Nacional_de_Referencia",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6686"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",4.59620041666667],PARAMETER["central_meridian",-74.0775079166667],PARAMETER["scale_factor",1],PARAMETER["false_easting",1000000],PARAMETER["false_northing",1000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(12.0, 0.0, 418881.0,
       0.0, -12.0, 1898535.0)}
Datos del Sentinel 2
 {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 1741, 'height': 18

## Analisis inicial de datos
1. Cortar el DEM de Colombia con el límite de la cuenca
2. Extrar las bandas de Sentinel 2 y calcular indices:
   - SAVI
   - BSI
   - SWI
   - NDMI

In [12]:
# Importe de librerias
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
from pathlib import Path
import pandas as pd
from shapely.geometry import Point
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.patches import Patch
import numpy as np
import shapely
import rasterio
import rasterio.plot
from rasterio.mask import mask
from shapely.geometry import mapping
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [15]:
#Rutas para cargar archivos shape
SHP=r"/notebooks/Insumos_Proyecto_Clase/"
geom_ruta=SHP+r"Geomorfologia.shp"
geol_ruta=SHP+r"Geologia.shp"
clim_ruta=SHP+r"Clima_IGAC.shp"
limBuf_ruta=SHP+r"Limite_Buff_500.shp"
# Carga de archivos shape y proyeccion a EPSG 9377
geom_shp = gpd.read_file(geom_ruta)
geom_shp = geom_shp.to_crs(9377)
    
geol_shp = gpd.read_file(geol_ruta)
geol_shp = geol_shp.to_crs(9377)
    
clim_shp = gpd.read_file(clim_ruta)
clim_shp = clim_shp.to_crs(9377)
    
limBuf_shp = gpd.read_file(limBuf_ruta)
limBuf_shp = limBuf_shp.to_crs(9377)

In [21]:
RAS=r"/notebooks/Insumos_Proyecto_Clase/"
Sentinel_ruta=RAS+r"Sentinel2.tif"
Dem_ruta=RAS+r"Dem_alos_125.tif"
    
#Carga de raster
sentinel2 = rasterio.open(Sentinel_ruta)
dem = rasterio.open(Dem_ruta)

# Abrir el dem
dem = rasterio.open(Dem_ruta)
# Verificar si el dem y el limite tienen el mismo sistema coordenado. Si no son iguales, reporyectar el limite al del dem
if limBuf_shp.crs != dem.crs:
    limBuf_shp = limBuf_shp.to_crs(dem.crs)
    
# Para cortar el DEM se convierte la geometria de limite a Json
#geoms = [feature["geometry"] for feature in limBuf_shp._geo_interface_["features"]]

geom = mapping(limBuf_shp.geometry.iloc[0])

print("CRS del DEM:\n", dem.crs)
print("\n\nCRS del límite:\n", limBuf_shp.crs)

valor_nodata = -9999
# Ejecutar la funcion de corte con mask y obtener en las nuevas variables los datos y el recorte
Dem_cortado, nueva_transform = mask(dem, [geom], crop=True, nodata=valor_nodata)

# Obtener los metadatos, se actualizan con los de el recorte y se cierra el uso de dem
metadatos = dem.meta.copy()


metadatos.update({
    "driver": "GTiff",
    "height": Dem_cortado.shape[1],
    "width": Dem_cortado.shape[2],
    "transform": nueva_transform,
    "nodata": valor_nodata
})

# Guardar el dem recortado
ruta_salida_nombre = "/notebooks/Proyecto_Clase/results/DEM_Cortado_Curiti_Buff.tif"
with rasterio.open(ruta_salida_nombre, "w", **metadatos) as guardar:
    guardar.write(Dem_cortado)
    
# Crear avisos para saber si se creó correctamente o no
if Path(ruta_salida_nombre).exists():
    print("✅ Archivo guardado correctamente.")
else:
    print("❌ No se guardó el archivo.")
   
# Cerrar el dem
print("Shape del DEM cortado:", Dem_cortado.shape)
print("Valores únicos:", np.unique(Dem_cortado))
dem.close()

CRS del DEM:
 PROJCS["Marco_Geocentrico_Nacional_de_Referencia_Transverse_Mercator",GEOGCS["GCS_Marco_Geocentrico_Nacional_de_Referencia",DATUM["Marco_Geocentrico_Nacional_de_Referencia",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6686"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",4.59620041666667],PARAMETER["central_meridian",-74.0775079166667],PARAMETER["scale_factor",1],PARAMETER["false_easting",1000000],PARAMETER["false_northing",1000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


CRS del límite:
 PROJCS["Marco_Geocentrico_Nacional_de_Referencia_Transverse_Mercator",GEOGCS["GCS_Marco_Geocentrico_Nacional_de_Referencia",DATUM["Marco_Geocentrico_Nacional_de_Referencia",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6686"]],PRIMEM["Greenwich",0],UNIT["degree"